# CSI4142 - Data Science

## Assignment 3 - Predictive analysis - Regression and Classification

Shacha Parker 300235525

Callum Frodsham 300199446

This part of the assignment takes a dataset and performs an empirical study with a linear regression task.

#### Execution of the Notebook

Execution of the notebook requires a recent version of Python and a virtual environment if your operating system requires it.

You also need the following libraries installed for import dependencies: numpy, pandas, seaborn, matplotlib, and sklearn.
### Dataset Information
Dataset name: <a href="https://www.kaggle.com/datasets/mirichoi0218/insurance">Medical Cost Personal Datasets</a>
<br>
Provider: Miri Choi on Kaggle

<b>Features</b>

    age: age of primary beneficiary - quantitative - continuous

    sex: insurance contractor gender, female, male - categorical, ordinal

    bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9 - quantitative - continuous

    children: Number of children covered by health insurance / Number of dependents - quantitative, discrete

    smoker: does the beneficiary smoke - categorical, nominal

    region: the beneficiary's residential area in the US: northeast, southeast, southwest, northwest. - categorical, nominal

    charges: Individual medical costs billed by health insurance - quantitative, continuous


In [ ]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Load the dataset
# CHANGE THIS TO GITHUB RAW
data = pd.read_csv("https://raw.githubusercontent.com/ShackuOttawa/Assignment3CSI4142/refs/heads/main/insurance.csv")

# output settings for debugging
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Data Cleaning
The dataset is already clean. This can be seen with the results of the code cell below: no entries are null, and all values are appropriate. No cleaning is needed.

In [ ]:
print(data.info())
print(data.describe())

# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Display unique values
categorical_columns = data.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\nUnique values in column '{column}':\n", data[column].unique())

## Categorical Feature Encoding
Through the get_dummies function, we can create the dataset with all categorical features becoming one-hot vectors. The rest of the features remain as they were.

In [ ]:
# One-hot encode the specified categorical features
data = pd.get_dummies(data, columns=['sex', 'smoker', 'region'], drop_first=False)

## EDA and Outlier detection
a. Outlier detection

In [ ]:
# Comparison graph for sex_male against sex_female
plt.figure(figsize=(10, 5))
sns.countplot(data=data, x='sex_male')
plt.title('Count of Male vs Female')
plt.xlabel('Sex (0: Female, 1: Male)')
plt.ylabel('Count')
plt.show()

# Comparison graph for smoker counts
plt.figure(figsize=(10, 5))
sns.countplot(data=data, x='smoker_yes')
plt.title('Count of Smokers vs Non-Smokers')
plt.xlabel('Smoker (0: No, 1: Yes)')
plt.ylabel('Count')
plt.show()

# Comparison for regions
plt.figure(figsize=(10, 5))
region_columns = ['region_northeast', 'region_northwest', 'region_southeast', 'region_southwest']
region_counts = data[region_columns].sum()
sns.barplot(x=region_counts.index, y=region_counts.values)
plt.title('Count of Regions')
plt.xlabel('Region')
plt.ylabel('Count')
plt.show()

Naturally, outliers don't occur on categorical data like this.

In [ ]:
# Visualize all numerical features
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
for column in numerical_columns:
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=data, x=column)
    plt.title(f'Box plot of {column}')
    plt.show()

Since feature 'bmi' has only 9 outliers out of the 1000+ entries (0.67%), we can safely remove them without there being a significant statistical impact.

In [ ]:
data_without_outlier_removal = data.copy()
original = data.shape

Q1 = data['bmi'].quantile(0.25)
Q3 = data['bmi'].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
data = data[(data['bmi'] >= lower_bound) & (data['bmi'] <= upper_bound)]

plt.figure(figsize=(10, 5))
sns.boxplot(data=data, x=data['bmi'])
plt.title(f'Box plot of BMI after outlier removal')
plt.show()

print("Original data shape:", original)
print("Data shape after removing outliers:", data.shape)

For the 'charges' outliers, it would negatively impact the dataset if all rows with 'charges' outliers were removed. Instead, we'll use  linear regression imputation to apply known values over the existing outlier values.

In [ ]:
D2 = data.copy()

# Create LocalOutlierFactor, fit it to temporary dataframe D2, and remove outliers
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
D2['charges_outlier'] = lof.fit_predict(D2[['charges']])

D2.loc[D2['charges_outlier'] == -1, 'charges'] = np.nan
D2.drop(columns=['charges_outlier'], inplace=True)

print("Number of NaN values in 'charges' after outlier removal:", D2['charges'].isnull().sum())

This imputation strategy is also a demonstration of my knowledge and exploration of sklearn's LinearRegression method for part d) Predictive analysis.

In [ ]:
# Separate the features and target variable in the original data
X_original = data.drop(columns=['charges'])
y_original = data['charges']

# Train a linear regression model on the original data
imputation_model = LinearRegression()
imputation_model.fit(X_original, y_original)

# Identify the rows in D2 with missing 'charges' values
missing_indices = D2[D2['charges'].isnull()].index
X_missing = D2.loc[missing_indices].drop(columns=['charges'])

# Replace missing values through imputation prediction
imputed_values = imputation_model.predict(X_missing)
D2.loc[missing_indices, 'charges'] = imputed_values

missing_values_d2_after_imputation = D2['charges'].isnull().sum()
print("Missing values in 'charges' column of D2 after imputation:", missing_values_d2_after_imputation)

# Return D2 values to main dataframe
data = D2.copy()

## Feature Engineering
Before we proceed with feature engineering, we'll create new copies of the dataset with and without outlier removal that will not have the extra features.

In [ ]:
data_without_outlier_removal_or_FE = data_without_outlier_removal.copy()
data_without_FE = data.copy()

For the first new feature, we will create the age-bmi column. The age-bmi value is a row's age multiplied by its bmi, which is then normalized by dividing this number by the median age times the median bmi.

In [ ]:
medianage = data['age'].median()
medianbmi = data['bmi'].median()
data['age-bmi'] = (data['age'] * data['bmi']) / (medianage * medianbmi)
data_without_outlier_removal_with_FE = data_without_outlier_removal
data_without_outlier_removal_with_FE['age-bmi'] = (data_without_outlier_removal['age'] * data_without_outlier_removal['bmi']) / (medianage * medianbmi)

Secondly, we'll create a feature called "childandsmoker" which is a boolean value that is true if the row has a positive number of children and 'smoker' is 'yes'.

In [ ]:
data['childandsmoker'] = np.where((data['smoker_yes'] == True) & (data['children'] > 0), True, False)
data_without_outlier_removal_with_FE['childandsmoker'] = np.where((data_without_outlier_removal_with_FE['smoker_yes'] == True) & (data_without_outlier_removal_with_FE['children'] > 0), True, False)

In [ ]:
# Renaming these variables to be more legible
data_with_both = data
data_with_neither = data_without_outlier_removal_or_FE
data_with_FE = data_without_outlier_removal_with_FE
data_with_OR = data_without_FE

At this point, we have:

*data_with_both*: the data with both outliers removed and new features engineered.

*data_with_neither*: data with neither outliers removed nor feature engineering

*data_with_FE*: data including outliers without feature engineering applied

*data_with_OR*: data with outliers removed but without feature engineering

## Empirical study

First, we split each dataset into train, validation, and test sets.

In [ ]:
# Split the data_with_both dataset
data_with_both_train, data_with_both_temp = train_test_split(data_with_both, test_size=0.3, random_state=0)
data_with_both_validation, data_with_both_test = train_test_split(data_with_both_temp, test_size=0.5, random_state=0)

# Split the data_with_neither dataset
data_with_neither_train, data_with_neither_temp = train_test_split(data_with_neither, test_size=0.3, random_state=0)
data_with_neither_validation, data_with_neither_test = train_test_split(data_with_neither_temp, test_size=0.5, random_state=0)

# Split the data_with_FE dataset
data_with_FE_train, data_with_FE_temp = train_test_split(data_with_FE, test_size=0.3, random_state=0)
data_with_FE_validation, data_with_FE_test = train_test_split(data_with_FE_temp, test_size=0.5, random_state=0)

# Split the data_with_OR dataset
data_with_OR_train, data_with_OR_temp = train_test_split(data_with_OR, test_size=0.3, random_state=0)
data_with_OR_validation, data_with_OR_test = train_test_split(data_with_OR_temp, test_size=0.5, random_state=0)

In [ ]:
# Function that 
def evaluate_model(X_train, y_train, X_val, y_val, info):
    # Initialize the linear regression model
    model = LinearRegression()

    # Perform 4-fold cross-validation on the training set
    kf = KFold(n_splits=4, shuffle=True, random_state=0)
    mse_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
    r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')

    # Fit and then train the model on the designated train set
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    # Calculate MSE and R2 on the validation set
    mse_val = mean_squared_error(y_val, y_val_pred)
    r2_val = r2_score(y_val, y_val_pred)

    # Print the results
    print("Results for", info)
    print("Cross-validated MSE scores:", -mse_scores)
    print("Cross-validated R2 scores:", r2_scores)
    print("Mean cross-validated MSE:", -mse_scores.mean())
    print("Mean cross-validated R2:", r2_scores.mean())
    print("Validation MSE:", mse_val)
    print("Validation R2:", r2_val)

#### Evaluation and Results

In [ ]:
X_train = data_with_neither_train.drop(columns=['charges'])
y_train = data_with_neither_train['charges']
X_val = data_with_neither_validation.drop(columns=['charges'])
y_val = data_with_neither_validation['charges']

evaluate_model(X_train, y_train, X_val, y_val, "data_with_neither")

In [ ]:
X_train = data_with_OR_train.drop(columns=['charges'])
y_train = data_with_OR_train['charges']
X_val = data_with_OR_validation.drop(columns=['charges'])
y_val = data_with_OR_validation['charges']

evaluate_model(X_train, y_train, X_val, y_val, "data_with_OR")

In [ ]:
X_train = data_with_FE_train.drop(columns=['charges'])
y_train = data_with_FE_train['charges']
X_val = data_with_FE_validation.drop(columns=['charges'])
y_val = data_with_FE_validation['charges']

evaluate_model(X_train, y_train, X_val, y_val, "data_with_FE")

In [ ]:
X_train = data_with_both_train.drop(columns=['charges'])
y_train = data_with_both_train['charges']
X_val = data_with_both_validation.drop(columns=['charges'])
y_val = data_with_both_validation['charges']

evaluate_model(X_train, y_train, X_val, y_val, "data_with_both")

It seems that the best performance is with the data with neither OR or FE applied, by a very small margin. We will use this now to run the test set.

In [ ]:
X_train = data_with_neither_train.drop(columns=['charges'])
y_train = data_with_neither_train['charges']
# Prepare the test data for data_with_neither
X_test_neither = data_with_neither_test.drop(columns=['charges'])
y_test_neither = data_with_neither_test['charges']

# Evaluate the model on the test dataset for data_with_neither
evaluate_model(X_train, y_train, X_test_neither, y_test_neither, "data_with_neither (Test Set)")

Let's now run the tests on the other models.

In [ ]:
X_train = data_with_FE_train.drop(columns=['charges'])
y_train = data_with_FE_train['charges']
X_test = data_with_FE_test.drop(columns=['charges'])
y_test = data_with_FE_test['charges']

# Evaluate the model on the test dataset for data_with_FE
evaluate_model(X_train, y_train, X_test, y_test, "data_with_FE (Test Set)")
print()

X_train = data_with_OR_train.drop(columns=['charges'])
y_train = data_with_OR_train['charges']
X_test_OR = data_with_OR_test.drop(columns=['charges'])
y_test_OR = data_with_OR_test['charges']

# Evaluate the model on the test dataset for data_with_OR
evaluate_model(X_train, y_train, X_test_OR, y_test_OR, "data_with_OR (Test Set)")
print()

X_train = data_with_both_train.drop(columns=['charges'])
y_train = data_with_both_train['charges']
X_test_both = data_with_both_test.drop(columns=['charges'])
y_test_both = data_with_both_test['charges']

# Evaluate the model on the test dataset for data_with_both
evaluate_model(X_train, y_train, X_test_both, y_test_both, "data_with_both (Test Set)")

## Result Analysis

Across the different dataset variations, the dataset with neither outlier removal nor feature engineering did indeed turn out to have the best scores (lowest MSE, highest R2) of the 4. It's worth noting that the difference in some scores was quite marginal. I tried the train_test_split again with a random value of 42 for all splits, and the feature engineering dataset was slightly higher, but almost in a near tie with the data_with_neither dataset in training/validation. However, when applied to testing, it turned out to be unexpectedly worse in scores than the other datasets. In my tests, I believe that outlier detection and removal generally worsened results slightly, while feature engineering had a neutral-to-positive effect, making it about equal to the dataset with neither technique applied. Interestingly, I found that in many tests, applying both techniques (data_with_both) consistently gave the worst scores of all. In the currently-set state, the results of the unseen tests lined up as expected with the cross-validation training/validations. I find this will typically be the case, but because the scores are so close to each other, it's possible some certain random configurations may cause unexpected results between the validation and the unseen tests.